In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from jupyter_dash import JupyterDash
from dash import dcc, html

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

education = pd.read_csv("../Data/states_all.csv").assign(
    expenditure_per_student = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"],
    above_avg_math8 = lambda x: np.where(x["AVG_MATH_8_SCORE"] > 278.6,'Above Avg', 'Below Avg')
)

education.tail()

/var/folders/bm/h1t1wfk16y5gv2b9svjm7mzw0000gn/T/ipykernel_48678/673105473.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,expenditure_per_student,above_avg_math8
1710,2019_VIRGINIA,VIRGINIA,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,247.0,287.0,224.0,262.0,NaN,Above Avg
1711,2019_WASHINGTON,WASHINGTON,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,240.0,286.0,220.0,266.0,NaN,Above Avg
1712,2019_WEST_VIRGINIA,WEST_VIRGINIA,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,231.0,272.0,213.0,256.0,NaN,Below Avg
1713,2019_WISCONSIN,WISCONSIN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,242.0,289.0,220.0,267.0,NaN,Above Avg
1714,2019_WYOMING,WYOMING,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,246.0,286.0,227.0,265.0,NaN,Above Avg


In [2]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.Dropdown(
        id="filter",
        options=[{"label": state.title(), "value": state} for state in education["STATE"].unique()
                ],
        value=["CALIFORNIA"],
        multi=True
   ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("filter", "value"),
)

def plot_rev_scatter(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016")
         .groupby(["YEAR", "STATE"])
         .sum()
         .reset_index()
        ), 
        x="YEAR", 
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title = f"Expenditure Over Time, Selected States",
    ) 
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline", debug=True)

/Users/pocholo/Desktop/projects/Python_Data_Visualization_Plotly_and_Dash/venv/lib/python3.11/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [3]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="filter",
        # options = ["CALIFORNIA", "TEXAS", "FLORIDA", "OREGON"],
        options=[{"label": state.title(), "value": state} for state in education["STATE"].unique()],
        value=["CALIFORNIA", "TEXAS"]
    ),
    dcc.Graph(id="graph")    
])

@app.callback(
    Output("graph", "figure"),
    Input("filter", "value")
)

def plot_rev_scatter(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016")
         .groupby(["YEAR", "STATE"])
         .sum()
         .reset_index()
        ),
        x="YEAR",
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title = f"Expenditure Over Time, Selected States"
    )
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port=4829)

Address already in use
Port 4829 is in use by another program. Either identify and stop that program, or start the server with a different port.


OSError: Address 'http://127.0.0.1:4829' already in use.
    Try passing a different port to run_server.

In [4]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id="aggfunc",
        options=["mean", "sum", "median"],
        value="mean"
    ),
    dcc.Graph(id="graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("aggfunc", "value")
)

def plot_rev_scatter(metric):
    fig = px.bar(
        (education
        .query("STATE in ['CALIFORNIA', 'OREGON', 'WASHINGTON', 'IDAHO'] and 1992 < YEAR < 2016")
        .groupby(["STATE"])
        .agg({"TOTAL_EXPENDITURE": f"{metric}"})
        .reset_index()
        ),
        x="STATE",
        y="TOTAL_EXPENDITURE",
        text_auto=True,
        title = f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=8274)

Address already in use
Port 8274 is in use by another program. Either identify and stop that program, or start the server with a different port.


OSError: Address 'http://127.0.0.1:8274' already in use.
    Try passing a different port to run_server.

In [5]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.Slider(
       id="score_slider",
       min=260,
       max=300,
       step=5,
       value=education["AVG_MATH_8_SCORE"].mean()
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("score_slider", "value"),
)

def plot_rev_scatter(score):
    df = (
        education
        .query("YEAR == 2013")
        .assign(
            score_cutoff =  lambda x: np.where(
                x["AVG_MATH_8_SCORE"] > score,
                'Above Threshold', 
                'Below Threshold'
            )
        )
    )
    fig = px.scatter(
        df,
        x="AVG_MATH_8_SCORE", 
        y="expenditure_per_student",
        color="score_cutoff",
        hover_name="STATE",
        title = f"Score and Expenditure vs. Threshold",
        )
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline",debug=True, port=2934)

Address already in use
Port 2934 is in use by another program. Either identify and stop that program, or start the server with a different port.


OSError: Address 'http://127.0.0.1:2934' already in use.
    Try passing a different port to run_server.

In [6]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.Slider(
       id="score_slider",
       min=260,
       max=300,
       step=5,
       value=education["AVG_MATH_8_SCORE"].mean()
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("score_slider", "value"),
)

def plot_rev_scatter(score):
    df = (
        education
        .query("YEAR == 2013")
        .assign(
            score_cutoff =  lambda x: np.where(
                x["AVG_MATH_8_SCORE"] > score,
                'Above Threshold', 
                'Below Threshold'
            )
        )
    )
    fig = px.scatter(
        df,
        x="AVG_MATH_8_SCORE", 
        y="expenditure_per_student",
        color="score_cutoff",
        hover_name="STATE",
        title = f"Score and Expenditure vs. Threshold",
        )
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline",debug=True, port=2934)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [7]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.RangeSlider(
        id="score_slider",
        min=education.query("YEAR == 2013")["AVG_MATH_8_SCORE"].min(),
        max=education.query("YEAR == 2013")["AVG_MATH_8_SCORE"].max(),
        step=5,
        value=[270, 280]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("score_slider", "value"),
)

def plot_rev_scatter(score):
    df = (
        education
        .query("YEAR == 2013")
        .assign(
            score_cutoff =  lambda x: np.where(
                x["AVG_MATH_8_SCORE"].between(score[0], score[1]),
                'In Range', 
                'Out of Range'
            )
        )
    )
    fig = px.scatter(
        df,
        x="AVG_MATH_8_SCORE", 
        y="expenditure_per_student",
        color="score_cutoff",
        hover_name="STATE",
        title = f"Score and Expenditure vs. Threshold",
        )
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline",debug=True, port=2934)

TypeError: 'NoneType' object cannot be interpreted as an integer